In [30]:
import time
import random
import math
import pandas as pd
from tqdm import tqdm
from functools import partial
from matplotlib import pyplot as plt

In [24]:
# from time import ctime
# ctime()

'Thu Dec  9 11:24:38 2021'

In [25]:
## Add operation
add_op = 1
count=0
st_time = time.time()
for i in range(2, (10**6)+1):
    add_op = i + add_op
time_taken = time.time() - st_time
print(f"Total time taken for add operation: {time_taken} seconds")
print(f"Approximate time taken for one add operation: {(time_taken / (10**6))} seconds")
print(add_op)
# print(count)

Total time taken for add operation: 0.46225810050964355 seconds
Approximate time taken for one add operation: 4.622581005096436e-07 seconds
500000500000


In [26]:
## Multiplication operation
mul_op = 1
st_time = time.time()
for i in range(2, (10**6)+1):
    mul_op = i * mul_op
time_taken = time.time() - st_time
print(f"Time taken for multiplication operation: {time_taken} seconds")
print(f"Approximate time taken for one multiplication operation: {time_taken / (10**6)} seconds")


Time taken for multiplication operation: 1798.6975429058075 seconds
Approximate time taken for one multiplication operation: 0.0017986975429058075 seconds


In [27]:
## Division operation
div_op = 1
st_time = time.time()
for i in tqdm(range(2, (10**6)+1)):
    div_op = i / div_op
time_taken = time.time() - st_time
print(f"Time taken for division operation: {time_taken} seconds")
print(f"Approximate time taken for one division operation: {time_taken / (10**6)} seconds")
print(div_op)


100%|██████████████████████████████████████████████████████████████████████| 999999/999999 [00:01<00:00, 622363.20it/s]

Time taken for division operation: 2.5107297897338867 seconds
Approximate time taken for one division operation: 2.5107297897338866e-06 seconds
1253.3144506440922


In [31]:
def roundto_d_significant(value, sig_digits):
    if value == 0:
        return 0
    return round(value, sig_digits - int(math.floor(math.log10(abs(value)))) - 1)


def rank_of_matrix(matrix):
    rank = 0
    for row in matrix:
        if sum(row) != 0:
            rank += 1
    return rank
    

def guass_elimination(A: list, b: list, partial_pivoting: bool = False, d: int = 3):
    """
    A: A square matrix n*n
    b: A vecor of size n
    partial_pivoting: Whether to perform partial pivoting or not
    d: number of significant digits to be rounded to
    """
    assert len(A) == len(b) # No of equations is equal to length of vector
    for row in A:
        assert len(row) == len(A) # Assert square matrix
        
    rounding = partial(roundto_d_significant, sig_digits = d)
    
    # Create augmented matrix
    for i in range(len(A)):
        A[i].append(b[i])
    
    # Bringing the matrix to reduced echelon form (REF)
    for index_row in range(0, len(A)-1):
        
        # Partial pivoting
        if partial_pivoting:
            cur_max = abs(A[index_row][index_row])
            cur_max_row = index_row
            # Check if pivoting need to be done
            for i in range(index_row + 1, len(A)):
                if abs(A[i][index_row]) > cur_max:
                    cur_max = abs(A[i][index_row]) # Absolute value considered for pivoting
                    cur_max_row = i
            # If pivoting need to be done
            if cur_max_row != index_row:
                temp = A[index_row]
                A[index_row] = A[cur_max_row]
                A[cur_max_row] = temp
        
        for i in range(index_row + 1, len(A)):
            # Skip the rwo transform if the value is already zero
            if A[i][index_row] == 0:
                continue
            
            # Calculate the coefficient to mutiply with the index row
            scaler = rounding(A[i][index_row] / A[index_row][index_row])
            if (A[i][index_row] < 0 and A[index_row][index_row] < 0) or ((A[i][index_row] > 0 and A[index_row][index_row] > 0)):
                scaler = -scaler
            
            # Assign all the prior values to zero
            for j in range(0, index_row + 1):
                A[i][j] = 0

            # Compute rest of the values in the row
            for j in range(index_row + 1, len(A[i])):
                A[i][j] = rounding(A[i][j] + (scaler * A[index_row][j]))
                
                    
    # Back substitution

    # Initialize None for all variables
    variable_values = {}
    for i in range(len(A)):
        variable_values[f'x{i}'] = None
    
    # Assign arbitrary values if needed
    no_aug_A = [[A[i][j] for j in range(len(A[i]) - 1)] for i in range(len(A))]
    if rank_of_matrix(no_aug_A) != rank_of_matrix(A):
        print("The system is inconsistent!!")
        return
    else:
        if rank_of_matrix(A) < len(A):
            for i in range(len(A)-1, rank_of_matrix(A)-1, -1):
                variable_values[f'x{i}'] = rounding(random.random())
        
        # Solve different equations for different variables
        for i in range(rank_of_matrix(no_aug_A) - 1, -1, -1):
            known_coeffs = [A[i][j] * variable_values[f'x{j}'] for j in range(i+1, len(A[i]) - 1)]
            known_coeffs = [rounding(v) for v in known_coeffs]
            rhs = rounding((A[i][-1] - rounding(sum(known_coeffs))))
            variable_values[f'x{i}'] =  rounding(rhs / A[i][i])

    return variable_values


In [32]:
guass_elimination(A=[[1, 1, 1], [2, 4, 1], [6, 10, 4]], b=[3, 8, 22], partial_pivoting=False, d=4)


{'x0': 1.602, 'x1': 1.133, 'x2': 0.2646}

In [33]:
def guass_operation_count(n: int):
    """
    n: Number of row / columns in a square matrix
    """
    # Operation count for addition
    ref_addition = (n * (n+1) * (2 * n + 1)) / 6 # Number of addition for REF
    backsub_addition = (n * (n - 1)) / 2
    total_addition = ref_addition + backsub_addition
    
    # Operation count for multiplication
    total_multiplication = total_addition # Same as addition
    
    # Operation count for division
    ref_division = (n * (n-1)) / 2
    backsub_division = n
    total_division = ref_division + backsub_division

    # Summarize
    operation_count = {
        'addition': total_addition,
        'multiplication': total_multiplication,
        'division': total_division
    }
    
    return operation_count
    

In [34]:
guass_operation_count(3)


{'addition': 17.0, 'multiplication': 17.0, 'division': 6.0}

In [35]:
def generate_test_data():
    test_cases = {}
    for case in range(100, 1100, 100):
        A = [[roundto_d_significant(random.random(), sig_digits=5) for _ in range(case)]for _ in range(case)]
        b = [roundto_d_significant(random.random(), sig_digits=5) for _ in range(case)]
        test_cases[case] = {'A': A, 'b': b}

    return test_cases
    

In [36]:
test_suit = generate_test_data()
for each_case in test_suit.items():
    print(each_case[0])
    print(f"A: {len(each_case[1]['A'])} * {len(each_case[1]['A'][0])}")
    print(f"b: {len(each_case[1]['b'])}")
    print("-"*30)
    

100
A: 100 * 100
b: 100
------------------------------
200
A: 200 * 200
b: 200
------------------------------
300
A: 300 * 300
b: 300
------------------------------
400
A: 400 * 400
b: 400
------------------------------
500
A: 500 * 500
b: 500
------------------------------
600
A: 600 * 600
b: 600
------------------------------
700
A: 700 * 700
b: 700
------------------------------
800
A: 800 * 800
b: 800
------------------------------
900
A: 900 * 900
b: 900
------------------------------
1000
A: 1000 * 1000
b: 1000
------------------------------


In [ ]:
time_taken = {}
for case_key, each_case in test_suit.items():
    print(case_key)
    st_time = time.time()
    solution = guass_elimination(A = deepcopy(each_case['A']), b = deepcopy(each_case['b']), partial_pivoting=True, d=5)
    time_taken = time.time() - st_time
    each_case['with_pivoting_solution'] = solution
    each_case['with_pivoting_time_taken'] = time_taken
    print(each_case['with_pivoting_time_taken'])
    print("+"*30, "\n\n")
    

100
4.269088268280029
++++++++++++++++++++++++++++++ 


200
31.920185804367065
++++++++++++++++++++++++++++++ 


300
87.29261469841003
++++++++++++++++++++++++++++++ 


400
165.16228914260864
++++++++++++++++++++++++++++++ 


500
326.83081793785095
++++++++++++++++++++++++++++++ 


600
444.676739692688
++++++++++++++++++++++++++++++ 


700
722.1010580062866
++++++++++++++++++++++++++++++ 


800
1330.1994743347168
++++++++++++++++++++++++++++++ 


900
1864.9360291957855
++++++++++++++++++++++++++++++ 


1000


In [ ]:
time_taken = {}
for case_key, each_case in test_suit.items():
    print(case_key)
    st_time = time.time()
    solution = guass_elimination(A = deepcopy(each_case['A']), b = deepcopy(each_case['b']), partial_pivoting=False, d=5)
    time_taken = time.time() - st_time
    each_case['without_pivoting_solution'] = solution
    each_case['without_pivoting_time_taken'] = time_taken
    print(each_case['without_pivoting_time_taken'])
    print("+"*30, "\n\n")
    

In [ ]:
for case_id, each_case in test_suit.items():
    print(case_id)
    print(guass_operation_count(n=case_id))
    print("="*30, "\n")


In [ ]:
efficiency = {'test_case': [], 'with_pivoting_time_taken': [], 'without_pivoting_time_taken': [], 
              'theoritical_time': []}
for k, v in test_suit.items():
    efficiency['test_case'].append(k)
    efficiency['with_pivoting_time_taken'].append(v['with_pivoting_time_taken'])
    efficiency['without_pivoting_time_taken'].append(v['without_pivoting_time_taken'])
    theoritical_time = guass_operation_count(n=k)
    efficiency['theoritical_time'].append(theoritical_time['addition'] + theoritical_time['multiplication'] + theoritical_time['division'])
    

efficiency_df = pd.DataFrame(efficiency)
efficiency_df
    

In [ ]:
efficiency_df['log_n'] = efficiency_df['test_case'].map(math.log)
efficiency_df

In [ ]:
def calculate_slope(x: list, y: list):
    mean_x = sum(x) / len(x)
    mean_y = sum(y) / len(y)
    
    numerator = sum([(i-mean_x) * (j-mean_y) for i,j in zip(x, y)])
    dinominator = sum([(i-mean_x)**2 for i in x])
    
    return numerator / dinominator


def calculate_y_intercept(mean_x, mean_y, m):
    return mean_y - (m * mean_x)
    
    

In [ ]:
efficiency_df['log_with_pivoting_time_taken'] = efficiency_df['with_pivoting_time_taken'].map(math.log)

efficiency_df


In [ ]:
min_scale = min(efficiency_df['log_n'].tolist() + efficiency_df['log_with_pivoting_time_taken'].tolist())
max_scale = max(efficiency_df['log_n'].tolist() + efficiency_df['log_with_pivoting_time_taken'].tolist())

efficiency_df.plot.scatter(x = 'log_n', y = 'log_with_pivoting_time_taken', grid = True,
                           xlim = (min_scale, max_scale), ylim = (min_scale, max_scale));


In [ ]:
# Calculating slope and y-intercept
m = calculate_slope(x=efficiency_df['log_n'], y=efficiency_df['log_with_pivoting_time_taken'])
y_intercept = calculate_y_intercept(mean_x=efficiency_df['log_n'].mean(), 
                                    mean_y=efficiency_df['log_with_pivoting_time_taken'].mean(),
                                    m=m)
# Rounding off
m = roundto_d_significant(m, sig_digits=5)
y_intercept = roundto_d_significant(y_intercept, sig_digits=5)

# Fit the line and plot
efficiency_df['log_with_pivoting_fitted_y'] = efficiency_df['log_n'].map(lambda x: m * x + y_intercept)
ax = efficiency_df.plot.scatter(x = 'log_n', y = 'log_with_pivoting_time_taken', grid = True,
                                xlim = (min_scale, max_scale), ylim = (min_scale, max_scale), 
                                title = f"Slope: {m}")
efficiency_df.plot(x = 'log_n', y = 'log_with_pivoting_fitted_y', color='red' ,legend=False, ax=ax);
